In [ ]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.data_sources import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
candles = await clob.get_candles_last_days("binance", "BTC-USDT", "1m", 10)

In [ ]:
from core.features.candles.mean_reversion_channel import MeanReversionChannel, MeanReversionChannelConfig

config = MeanReversionChannelConfig(length=200)

candles.add_feature(MeanReversionChannel(config))

In [ ]:
df = candles.data

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from core.data_structures.candles import Candles
from core.features.candles.mean_reversion_channel import MeanReversionChannel, MeanReversionChannelConfig

def plot_mean_reversion_channel(candles: Candles, candles_with_mrc: pd.DataFrame):
    # Create the figure with two rows
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        specs=[[{"secondary_y": False}], [{"secondary_y": True}]]
    )

    # Add candlestick chart to the first row
    candlestick = candles.candles_trace()
    fig.add_trace(candlestick, row=1, col=1)

    # Add Mean Line to the first row
    fig.add_trace(
        go.Scatter(
            x=candles_with_mrc.index,
            y=candles_with_mrc['meanline'],
            mode='lines',
            name='Mean Line',
            line=dict(color="#FFCD00", width=2)
        ),
        row=1, col=1
    )

    # Add Upper and Lower Bands
    for band, color in [('upband1', 'green'), ('loband1', 'green'), ('upband2', 'red'), ('loband2', 'red')]:
        fig.add_trace(
            go.Scatter(
                x=candles_with_mrc.index,
                y=candles_with_mrc[band],
                mode='lines',
                name=f'{band.capitalize()}',
                line=dict(color=color, width=1),
                opacity=0.5
            ),
            row=1, col=1
        )

    # Add colored zones
    colors = ['#FF0000', '#FF4200', '#FF5D00', '#FF7400', '#FF9700', '#FFAE00', '#FFC500', '#FFCD00']
    for i in range(1, 10):
        color_index = min(i-1, len(colors)-1)  # Ensure we don't go out of bounds
        fig.add_trace(
            go.Scatter(
                x=candles_with_mrc.index,
                y=candles_with_mrc[f'upband2_{i}'],
                mode='lines',
                name=f'Upper Zone {i}',
                line=dict(color=colors[color_index], width=0),
                fill='tonexty',
                fillcolor=colors[color_index],
                opacity=0.1
            ),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(
                x=candles_with_mrc.index,
                y=candles_with_mrc[f'loband2_{i}'],
                mode='lines',
                name=f'Lower Zone {i}',
                line=dict(color=colors[color_index], width=0),
                fill='tonexty',
                fillcolor=colors[color_index],
                opacity=0.1
            ),
                row=1, col=1
        )
    
    # Add the condition to the second row
    fig.add_trace(
        go.Scatter(
            x=candles_with_mrc.index,
            y=candles_with_mrc['condition'],
            mode='lines',
            name='Condition',
            line=dict(color="#00FFFF", width=2)
        ),
        row=2, col=1
    )

    # Add MTF analysis
    for tf in ['', 'mtf1_', 'mtf2_']:
        if f'{tf}meanline' in candles_with_mrc.columns:
            fig.add_trace(
                go.Scatter(
                    x=candles_with_mrc.index,
                    y=candles_with_mrc[f'{tf}meanline'],
                    mode='lines',
                    name=f'{tf.upper()}Mean Line',
                    line=dict(dash='dash'),
                    visible='legendonly'
                ),
                row=1, col=1
            )

    # Update layout for dark theme and technical colors
    fig.update_layout(
        height=800,
        width=1800,
        title_text="Mean Reversion Channel with Multiple Timeframe Analysis",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        hovermode="x unified",
        hoverdistance=100,
        spikedistance=1000,
    )

    # Update y-axis labels and colors
    fig.update_yaxes(title_text="Price", row=1, col=1, color="#FFFFFF")
    fig.update_yaxes(title_text="Condition", row=2, col=1, color="#00FFFF")

    # Update x-axis color and add spikes
    fig.update_xaxes(
        color="#FFFFFF",
        showspikes=True,
        spikecolor="white",
        spikesnap="cursor",
        spikemode="across"
    )

    return fig

# Usage example:
config = MeanReversionChannelConfig(length=200)
candles.add_feature(MeanReversionChannel(config))

# Assuming you have implemented the MTF analysis in your MeanReversionChannel class
df = candles.data

# Calculate additional zone levels
for i in range(1, 10):
    df[f'upband2_{i}'] = df['upband2'] + (df['meanrange'] * 0.5 * (4 - i))
    df[f'loband2_{i}'] = df['loband2'] - (df['meanrange'] * 0.5 * (4 - i))

fig = plot_mean_reversion_channel(candles, df)
fig.show()